# Train simple models for prediction of In-hospital mortality

- Logistic regression
- Random forest
- XGBoost

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [144]:
from fastai.structured import *
from fastai.column_data import *
import yaml
from pandas import DataFrame, Series
import shutil

In [3]:
NB_DIR = %pwd

In [4]:
RAW_DATA = '/data1/MIMIC-III/RAW/'
INTERIM_DATA = f'{RAW_DATA}/../interim/'
PROCESSED_DATA = f'{RAW_DATA}/../processed/'

In [5]:
MIMIC3_BENCHMARK_LOCATION = f'{NB_DIR}/../mimic3-benchmarks/'

# Train-validation split

In [6]:
task='in-hospital-mortality'

In [7]:
val_patients = set()
with open(f"{MIMIC3_BENCHMARK_LOCATION}/mimic3models/valset.csv", "r") as valset_file:
    for line in valset_file:
        x, y = line.split(',')
        if int(y) == 1:
            val_patients.add(x)

In [8]:
with open(f"{PROCESSED_DATA}/{task}/train/listfile.csv") as listfile:
    lines = listfile.readlines()
    header = lines[0]
    lines = lines[1:]

In [9]:
train_lines = [x for x in lines if x[:x.find("_")] not in val_patients]
val_lines = [x for x in lines if x[:x.find("_")] in val_patients]
assert len(train_lines) + len(val_lines) == len(lines)

In [10]:
with open(f"{PROCESSED_DATA}/{task}/train_listfile.csv", "w") as train_listfile:
    train_listfile.write(header)
    for line in train_lines:
        train_listfile.write(line)

In [11]:
with open(f"{PROCESSED_DATA}/{task}/val_listfile.csv", "w") as val_listfile:
    val_listfile.write(header)
    for line in val_lines:
        val_listfile.write(line)

In [12]:
shutil.copy(f"{PROCESSED_DATA}/{task}/test/listfile.csv",
            f"{PROCESSED_DATA}/{task}/test_listfile.csv")

'/data1/MIMIC-III/RAW//../processed//in-hospital-mortality/test_listfile.csv'

# Logistic regression

In [13]:
from sklearn.preprocessing import Imputer, StandardScaler

## Read and extract features

In [14]:
def read_chunk(reader, chunk_size):
    data = {}
    for i in range(chunk_size):
        ret = reader.read_next()
        for k, v in iter(ret.items()):
            if k not in data:
                data[k] = []
            data[k].append(v)
    data["header"] = data["header"][0]
    return data

In [15]:
def extract_features_from_rawdata(chunk, header, period, features):
    with open(os.path.join(f'{MIMIC3_BENCHMARK_LOCATION}/resources/', "channel_info.json")) as channel_info_file:
        channel_info = json.loads(channel_info_file.read())
    data = [convert_to_dict(X, header, channel_info) for X in chunk]
    return extract_features(data, period, features)

In [52]:
def convert_to_dict(data, header, channel_info):
    """ convert data from readers output in to array of arrays format """
    ret = [[] for i in range(data.shape[1] - 1)]
    for i in range(1, data.shape[1]):
        ret[i-1] = [(t, x) for (t, x) in zip(data[:, 0], data[:, i]) if x != ""]
        channel = header[i]
        if (len(channel_info[channel]['possible_values']) != 0):
            ret[i-1] = list(map(lambda x: (x[0], channel_info[channel]['values'][x[1]]), ret[i-1])) # list(..) for Python3
        ret[i-1] = list(map(lambda x: (float(x[0]), float(x[1])), ret[i-1]))
    return ret

In [53]:
import numpy as np
from scipy.stats import skew

all_functions = [min, max, np.mean, np.std, skew, len]

functions_map = {
    "all": all_functions,
    "len": [len],
    "all_but_len": all_functions[:-1]
}

periods_map = {
    "all": (0, 0, 1, 0),
    "first4days": (0, 0, 0, 4 * 24),
    "first8days": (0, 0, 0, 8 * 24),
    "last12hours": (1, -12, 1, 0),
    "first25percent": (2, 25),
    "first50percent": (2, 50)
}

sub_periods = [(2, 100), (2, 10), (2, 25), (2, 50),
               (3, 10), (3, 25), (3, 50)]


def get_range(begin, end, period):
    # first p %
    if period[0] == 2:
        return (begin, begin + (end - begin) * period[1] / 100.0)
    # last p %
    if period[0] == 3:
        return (end - (end - begin) * period[1] / 100.0, end)

    if period[0] == 0:
        L = begin + period[1]
    else:
        L = end + period[1]

    if period[2] == 0:
        R = begin + period[3]
    else:
        R = end + period[3]

    return (L, R)


def calculate(channel_data, period, sub_period, functions):
    if len(list(channel_data)) == 0:
        return np.full((len(functions, )), np.nan)

    L = channel_data[0][0]
    R = channel_data[-1][0]
    L, R = get_range(L, R, period)
    L, R = get_range(L, R, sub_period)

    data = [x for (t, x) in channel_data
            if L - 1e-6 < t < R + 1e-6]

    if len(data) == 0:
        return np.full((len(functions, )), np.nan)
    return np.array([fn(data) for fn in functions], dtype=np.float32)


def extract_features_single_episode(data_raw, period, functions):
    global sub_periods
    extracted_features = [np.concatenate([calculate(data_raw[i], period, sub_period, functions)
                                          for sub_period in sub_periods],
                                         axis=0)
                          for i in range(len(data_raw))]
    return np.concatenate(extracted_features, axis=0)


def extract_features(data_raw, period, features):
    period = periods_map[period]
    functions = functions_map[features]
    return np.array([extract_features_single_episode(x, period, functions)
                     for x in data_raw])

In [18]:
def read_and_extract_features(reader, period, features):
    ret = read_chunk(reader, reader.get_number_of_examples())
    # ret = common_utils.read_chunk(reader, 100)
    X = extract_features_from_rawdata(ret['X'], ret['header'], period, features)
    return (X, ret['y'], ret['name'])

## Model

In [19]:
period = "all" # choices=['first4days', 'first8days', 'last12hours', 'first25percent', 'first50percent', 'all']
features = "all" # choices=['all', 'len', 'all_but_len']

### Read data

In [21]:
class Reader(object):
    def __init__(self, dataset_dir, listfile=None):
        self._dataset_dir = dataset_dir
        self._current_index = 0
        if listfile is None:
            listfile_path = os.path.join(dataset_dir, "listfile.csv")
        else:
            listfile_path = listfile
        with open(listfile_path, "r") as lfile:
            self._data = lfile.readlines()
        self._listfile_header = self._data[0]
        self._data = self._data[1:]

    def get_number_of_examples(self):
        return len(self._data)

    def random_shuffle(self, seed=None):
        if (seed is not None):
            random.seed(seed)
        random.shuffle(self._data)

    def read_example(self, index):
        raise NotImplementedError()

    def read_next(self):
        to_read_index = self._current_index
        self._current_index += 1
        if (self._current_index == self.get_number_of_examples()):
            self._current_index = 0
        return self.read_example(to_read_index)

In [22]:
class InHospitalMortalityReader(Reader):
    def __init__(self, dataset_dir, listfile=None, period_length=48.0):
        """ Reader for in-hospital moratality prediction task.
        :param dataset_dir:   Directory where timeseries files are stored.
        :param listfile:      Path to a listfile. If this parameter is left `None` then
                              `dataset_dir/listfile.csv` will be used.
        :param period_length: Length of the period (in hours) from which the prediction is done.
        """
        Reader.__init__(self, dataset_dir, listfile)
        self._data = [line.split(',') for line in self._data]
        self._data = [(x, int(y)) for (x, y) in self._data]
        self._period_length = period_length

    def _read_timeseries(self, ts_filename):
        ret = []
        with open(os.path.join(self._dataset_dir, ts_filename), "r") as tsfile:
            header = tsfile.readline().strip().split(',')
            assert header[0] == "Hours"
            for line in tsfile:
                mas = line.strip().split(',')
                ret.append(np.array(mas))
        return (np.stack(ret), header)

    def read_example(self, index):
        """ Reads the example with given index.
        :param index: Index of the line of the listfile to read (counting starts from 0).
        :return: Dictionary with the following keys:
            X : np.array
                2D array containing all events. Each row corresponds to a moment.
                First column is the time and other columns correspond to different
                variables.
            t : float
                Length of the data in hours. Note, in general, it is not equal to the
                timestamp of last event.
            y : int (0 or 1)
                In-hospital mortality.
            header : array of strings
                Names of the columns. The ordering of the columns is always the same.
            name: Name of the sample.
        """
        if (index < 0 or index >= len(self._data)):
            raise ValueError("Index must be from 0 (inclusive) to number of lines (exclusive).")

        name = self._data[index][0]
        t = self._period_length
        y = self._data[index][1]
        (X, header) = self._read_timeseries(name)

        return {"X": X,
                "t": t,
                "y": y,
                "header": header,
                "name": name}


In [23]:
train_reader = InHospitalMortalityReader(dataset_dir=f'{PROCESSED_DATA}/in-hospital-mortality/train/',
                                             listfile=f'{PROCESSED_DATA}/in-hospital-mortality/train_listfile.csv',
                                             period_length=48.0)

val_reader = InHospitalMortalityReader(dataset_dir=f'{PROCESSED_DATA}/in-hospital-mortality/train/',
                                           listfile=f'{PROCESSED_DATA}/in-hospital-mortality/val_listfile.csv',
                                           period_length=48.0)

test_reader = InHospitalMortalityReader(dataset_dir=f'{PROCESSED_DATA}/in-hospital-mortality/test/',
                                            listfile=f'{PROCESSED_DATA}/in-hospital-mortality/test_listfile.csv',
                                            period_length=48.0)

In [24]:
train_reader.get_number_of_examples()

14681

In [25]:
train_reader?

Type:           InHospitalMortalityReader
String form:    <__main__.InHospitalMortalityReader object at 0x7fbe4f3372b0>
Docstring:      <no docstring>
Init docstring:
Reader for in-hospital moratality prediction task.
:param dataset_dir:   Directory where timeseries files are stored.
:param listfile:      Path to a listfile. If this parameter is left `None` then
                      `dataset_dir/listfile.csv` will be used.
:param period_length: Length of the period (in hours) from which the prediction is done.


In [64]:
print('Reading data and extracting features ...')
(train_X, train_y, train_names) = read_and_extract_features(train_reader, period, features)
(val_X, val_y, val_names) = read_and_extract_features(val_reader, period, features)
(test_X, test_y, test_names) = read_and_extract_features(test_reader, period, features)
print('  train data shape = {}'.format(train_X.shape))
print('  validation data shape = {}'.format(val_X.shape))
print('  test data shape = {}'.format(test_X.shape))

Reading data and extracting features ...
  train data shape = (14681, 714)
  validation data shape = (3222, 714)
  test data shape = (3236, 714)


In [65]:
#%debug

**SAVE train, val, test arrays**

In [68]:
np.save(f'{PROCESSED_DATA}/{task}/train_X', train_X)
np.save(f'{PROCESSED_DATA}/{task}/train_y', train_y)
np.save(f'{PROCESSED_DATA}/{task}/train_names', train_names)

np.save(f'{PROCESSED_DATA}/{task}/val_X', val_X)
np.save(f'{PROCESSED_DATA}/{task}/val_y', val_y)
np.save(f'{PROCESSED_DATA}/{task}/val_names', val_names)

np.save(f'{PROCESSED_DATA}/{task}/test_X', test_X)
np.save(f'{PROCESSED_DATA}/{task}/test_y', test_y)
np.save(f'{PROCESSED_DATA}/{task}/test_names', test_names)

In [154]:
train_X = np.load(f'{PROCESSED_DATA}/{task}/train_X.npy')
train_y = np.load(f'{PROCESSED_DATA}/{task}/train_y.npy')
train_names = np.load(f'{PROCESSED_DATA}/{task}/train_names.npy')

val_X = np.load(f'{PROCESSED_DATA}/{task}/val_X.npy')
val_y = np.load(f'{PROCESSED_DATA}/{task}/val_y.npy')
val_names = np.load(f'{PROCESSED_DATA}/{task}/val_names.npy')


test_X = np.load(f'{PROCESSED_DATA}/{task}/test_X.npy')
test_y = np.load(f'{PROCESSED_DATA}/{task}/test_y.npy')
test_names = np.load(f'{PROCESSED_DATA}/{task}/test_names.npy')

**TESTS (TMP)**

In [54]:
ret = read_chunk(train_reader, train_reader.get_number_of_examples())

In [55]:
len(ret['X'])

14681

In [56]:
type(ret['X'])

list

In [57]:
header = ret['header']

In [58]:
len(header)

18

In [59]:
channel_info_file = open(f'{MIMIC3_BENCHMARK_LOCATION}/resources/channel_info.json')
channel_info = json.loads(channel_info_file.read())

In [60]:
data = [convert_to_dict(X, header, channel_info) for X in ret['X']]

In [61]:
type(data)

list

In [ ]:
data[0]

In [63]:
len(data)

14681

# Logistic regression

In [155]:
from sklearn.linear_model import LogisticRegression

In [156]:
C = 0.001 # Inverse of L1 / L2 regularization
l2 = True 
l1 = False

## Imputing

In [157]:
imputer = Imputer(missing_values=np.nan, strategy='mean', axis=0,
                  verbose=0, copy=True)

In [158]:
imputer.fit(train_X)

Imputer(axis=0, copy=True, missing_values=nan, strategy='mean', verbose=0)

In [159]:
train_X = np.array(imputer.transform(train_X), dtype=np.float32)
val_X = np.array(imputer.transform(val_X), dtype=np.float32)

## Normalizing

In [160]:
scaler = StandardScaler()

In [161]:
scaler.fit(train_X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [162]:
train_X = scaler.transform(train_X)
val_X = scaler.transform(val_X)

## Define logreg-model

In [163]:
penalty = ("l2" if l2 else "l1")

In [164]:
penalty

'l2'

In [165]:
logreg = LogisticRegression(penalty=penalty, C=C)

In [166]:
logreg.fit(train_X, train_y)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Predict on train and valdata

In [167]:
train_prob = logreg.predict_proba(train_X)
val_prob = logreg.predict_proba(val_X)

## Metrics

Accuracy:

In [168]:
logreg.score(train_X, train_y)

0.8842721885430148

In [169]:
from sklearn import metrics

In [170]:
def print_metrics_binary(y_true, predictions, verbose=1):
    predictions = np.array(predictions)
    if (len(predictions.shape) == 1):
        predictions = np.stack([1 - predictions, predictions]).transpose((1, 0))

    cf = metrics.confusion_matrix(y_true, predictions.argmax(axis=1))
    if verbose:
        print("confusion matrix:")
        print(cf)
    cf = cf.astype(np.float32)

    acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
    prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
    prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
    rec0 = cf[0][0] / (cf[0][0] + cf[0][1])
    rec1 = cf[1][1] / (cf[1][1] + cf[1][0])
    auroc = metrics.roc_auc_score(y_true, predictions[:, 1])

    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, predictions[:, 1])
    auprc = metrics.auc(recalls, precisions)
    minpse = np.max([min(x, y) for (x, y) in zip(precisions, recalls)])

    if verbose:
        print("accuracy =", acc)
        print("precision class 0 =", prec0)
        print("precision class 1 =", prec1)
        print("recall class 0 =", rec0)
        print("recall class 1 =", rec1)
        print("AUC of ROC =", auroc)
        print("AUC of PRC =", auprc)
        print("min(+P, Se) =", minpse)

    return {"acc": acc,
            "prec0": prec0,
            "prec1": prec1,
            "rec0": rec0,
            "rec1": rec1,
            "auroc": auroc,
            "auprc": auprc,
            "minpse": minpse}

In [171]:
acc, prec0, prec1, rec0, rec1, auroc, auprc, minpse = print_metrics_binary(train_y, train_prob)

confusion matrix:
[[12218   476]
 [ 1223   764]]
accuracy = 0.8842722
precision class 0 = 0.90900975
precision class 1 = 0.61612904
recall class 0 = 0.96250194
recall class 1 = 0.38449925
AUC of ROC = 0.8646263339721424
AUC of PRC = 0.5495369335364912
min(+P, Se) = 0.5288509784244857


In [172]:
acc_v, prec0_v, prec1_v, rec0_v, rec1_v, auroc_v, auprc_v, minpse_v = print_metrics_binary(val_y, val_prob)

confusion matrix:
[[2665  121]
 [ 274  162]]
accuracy = 0.87740535
precision class 0 = 0.906771
precision class 1 = 0.5724382
recall class 0 = 0.95656854
recall class 1 = 0.37155962
AUC of ROC = 0.8455737073308879
AUC of PRC = 0.48928153761470383
min(+P, Se) = 0.5160550458715596


## Predict on test

In [173]:
test_X = np.array(imputer.transform(test_X), dtype=np.float32)

In [174]:
test_X = scaler.transform(test_X)

In [175]:
test_prob = logreg.predict_proba(test_X)

In [176]:
acc_t, prec0_t, prec1_t, rec0_t, rec1_t, auroc_t, auprc_t, minpse_t = print_metrics_binary(test_y, test_prob)

confusion matrix:
[[2746  116]
 [ 232  142]]
accuracy = 0.8924598
precision class 0 = 0.92209536
precision class 1 = 0.5503876
recall class 0 = 0.9594689
recall class 1 = 0.37967914
AUC of ROC = 0.848424122841437
AUC of PRC = 0.4743702838948688
min(+P, Se) = 0.4613333333333333


# Random forests

In [177]:
from sklearn.ensemble import RandomForestClassifier

In [178]:
rf = RandomForestClassifier(n_jobs=-1)

In [179]:
rf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [180]:
train_prob_rf = rf.predict_proba(train_X)

In [181]:
rf.score(train_X, train_y)

0.9874667938151352

In [182]:
rf.score(val_X, val_y)

0.8767846058348852

In [183]:
rf.score(test_X, test_y)

0.8899876390605687

In [184]:
cf = metrics.confusion_matrix(test_y, rf.predict_proba(test_X).argmax(axis=1))

In [185]:
cf

array([[2795,   67],
       [ 289,   85]])

In [186]:
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)

In [187]:
m.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [188]:
m.score(test_X, test_y)

0.8992583436341162

In [189]:
metrics.confusion_matrix(test_y, rf.predict_proba(test_X).argmax(axis=1))

array([[2795,   67],
       [ 289,   85]])

# XGBoost

In [190]:
from xgboost import XGBClassifier

In [191]:
model = XGBClassifier()

In [192]:
model.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [193]:
test_proba_xg = model.predict(test_X)

/home/alex/anaconda2/envs/fastai/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [194]:
from sklearn.metrics import accuracy_score

In [195]:
accuracy_score(test_y, test_proba_xg)

0.8983312731767614

In [196]:
metrics.confusion_matrix(test_y, model.predict_proba(test_X).argmax(axis=1))

array([[2797,   65],
       [ 264,  110]])

In [197]:
metrics_xg = print_metrics_binary(test_y, test_proba_xg)

confusion matrix:
[[2797   65]
 [ 264  110]]
accuracy = 0.8983313
precision class 0 = 0.9137537
precision class 1 = 0.62857145
recall class 0 = 0.9772886
recall class 1 = 0.29411766
AUC of ROC = 0.6357031282114524
AUC of PRC = 0.5021356379387354
min(+P, Se) = 0.29411764705882354
